In [15]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd
from datetime import datetime,timedelta
import ast

In [16]:
plate = 13
date_init = datetime(2020,7,1,19,57)

In [17]:
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(8)]

In [18]:
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [ ]:
dates.remove('0705_1957')

In [15]:
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [16]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [17]:
skeleton_docs=[sparse_to_doc(sparse_skel) for sparse_skel in dic_skels]

In [ ]:
nx_graph_pivot,pos_pivot=generate_nx_graph_from_skeleton(skeleton_docs[0]) 

In [37]:
# nx_graph_poss=[]
# for date in dates:
#     nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,plate,True,extension='_raw_aligned_skeleton.csv'),
#                                     converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))

In [26]:
nx_graph_poss=[]
for date in dates:
    nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,plate,True,extension='_raw_aligned_skeleton.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))
# nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(realign(skeleton_doc,nx_graph_pivot,pos_pivot,2,maxdist = 50,
#                                                               save=get_path(date,plate,True,extension='')))) for skeleton_doc in skeleton_docs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [27]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-1
downstream_graphs=[nx_graph_pruned[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,-1,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(nx_graph_pruned[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 6
number removed first phase 91
number removed second phase 554
4276 378.87464945546304
4276 586.4230554812797
4541 18.788294228055936
4541 138.70832707519762
4596 11.180339887498949
4596 78.77182237323191
7888 53.81449618829484
7888 371.06603185956
4467 32.64965543462902
4467 921.0917435304694
6361 395.9153950025182
6361 300.26987860922713
4609 6.0
4609 45.09988913511872
4417 74.10802925459562
4417 461.29925211298576
6138 10.44030650891055
6138 64.28841264178172
7532 39.35733730830886
7532 30.083217912982647
7232 56.08029957123981
7232 10.04987562112089
5701 9.486832980505138
5701 10.0
4588 3.0
4588 21.633307652783937
5803 15.132745950421556
5803 4.123105625617661
7305 13.601470508735444
7305 10.0
5877 5.0990195135927845
5877 89.02246907382428
7673 9.055385138137417
7673 455.7510285232497
2 0 [7917 6761]
2 1 [7925 6765]
i= 5
number removed first phase 110
number removed second phase 416
4178 518.0279915216937
4178 342.4456161202827
6154 8.0
6154 93.33809511662427
6878 10.1980390271

In [28]:
nx_graph_pruned=downstream_graphs
poss_aligned=downstream_poss

In [11]:
# for i, date in enumerate(dates):
#     sparse.save_npz(f'Data/skeleton_{date}_{plate}_full_clean',sparse.csc_matrix(skeleton_docs_cleaned[i]))

In [14]:
for i, date in enumerate(dates):
    from_nx_to_tab(nx_graph_pruned[i],poss_aligned[i]).to_csv(get_path(date,plate,True,extension='_full_labeled.csv'))
    from_nx_to_tab_matlab(nx_graph_pruned[i],poss_aligned[i]).to_csv(get_path(date,plate,True,extension='_full_labeled_matlab.csv'))
#     tabs[i].to_csv(get_path(date,plate,True,extension='_full.csv'))
    

In [29]:
for i, date in enumerate(dates):
    from_nx_to_tab(nx_graph_pruned[i],poss_aligned[i]).to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
    from_nx_to_tab_matlab(nx_graph_pruned[i],poss_aligned[i]).to_csv(f'Data/graph_{date}_{plate}_full_labeled_matlab.csv')
# for i, date in enumerate(dates):
#     tabs_labeled[i].to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
#     tabs[i].to_csv(f'Data/graph_{date}_{plate}_full.csv')

In [30]:
connections_growth_pattern = [whole_movement_identification(nx_graph_pruned[i],nx_graph_pruned[i+1],poss_aligned[i],poss_aligned[i+1]) for i in range(len(dates)-1)]
tip_origins = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]
# for i in range(len(tip_origins)):
#     connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(get_path(dates[i],plate,True,extension='_connection.csv'))

0.0
0.1375515818431912
0.2751031636863824
0.4126547455295736
0.5502063273727648
0.687757909215956
0.8253094910591472
0.9628610729023384
0.0
0.13020833333333334
0.2604166666666667
0.390625
0.5208333333333334
0.6510416666666666
0.78125
0.9114583333333334
0.0
0.12195121951219512
0.24390243902439024
0.36585365853658536
0.4878048780487805
0.6097560975609756
0.7317073170731707
0.8536585365853658
0.975609756097561
0.0
0.11750881316098707
0.23501762632197415
0.3525264394829612
0.4700352526439483
0.5875440658049353
0.7050528789659224
0.8225616921269095
0.9400705052878966
0.0
0.10964912280701754
0.21929824561403508
0.32894736842105265
0.43859649122807015
0.5482456140350878
0.6578947368421053
0.7675438596491229
0.8771929824561403
0.9868421052631579
0.0
0.10395010395010396
0.2079002079002079
0.31185031185031187
0.4158004158004158
0.5197505197505198
0.6237006237006237
0.7276507276507277
0.8316008316008316
0.9355509355509356
0.0
0.0946969696969697
0.1893939393939394
0.2840909090909091
0.378787878787

In [31]:
def connections_pixel_list_to_tab(origin_tips,pattern_growth):
    column_names = ['tip_origin',"nodes_from_tip", "growth_pattern"]
    tab = pd.DataFrame(columns=column_names)
    pattern_growth={tip : [[(pixel[0],pixel[1]) for pixel in branch] for branch in pattern_growth[tip]] for tip in pattern_growth.keys()}
    for tip in origin_tips.keys():
        new_line=pd.DataFrame({"tip_origin" : [tip],"nodes_from_tip" :[origin_tips[tip]], "growth_pattern" : [pattern_growth[tip]]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)
# for i in range(len(tip_origins)):
#     connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(get_path(dates[i],plate,True,extension='_connection.csv'))
for i in range(len(tip_origins)):
    connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(f'Data/graph_{dates[i]}_{plate}_connection.csv')

In [ ]:
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [ ]:
xbegin=14000
xend = 18000
ybegin = 8000
yend=16000
sub_nx_graphs = [sub_graph(nx_graph,poss_aligned[i],xbegin,xend,ybegin,yend) for i, nx_graph in enumerate(nx_graph_pruned)]
# sub_skeleton= [generate_skeleton(nx_graph,(xend-xbegin,yend-ybegin),(xbegin,ybegin)) for nx_graph in sub_nx_graphs]

In [13]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-2
downstream_graphs=[sub_nx_graphs[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,0,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(sub_nx_graphs[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 3
number removed first phase 19
number removed second phase 36
116 0 [16887  8355]
116 1 [16893  8366]
i= 2
number removed first phase 13
number removed second phase 27
121 0 [17317  9088]
121 1 [17312  9091]
121 2 [17311  9107]
i= 1
number removed first phase 12
number removed second phase 32
119 0 [17318  9090]
119 1 [17317  9088]
119 2 [17312  9091]
119 3 [17311  9107]


In [25]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
from util import get_path
from plotutil import show_im,overlap, show_im_rgb
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton
import networkx as nx
from copy import deepcopy
from sparse_util import dilate
from scipy.optimize import minimize

def node_dist(node1,node2,nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,tolerance,show=False,):
    #!!! assumed shape == 3000,4096
    sparse_cross1=sparse.dok_matrix((4*tolerance,4*tolerance), dtype=bool)
    sparse_cross2=sparse.dok_matrix((4*tolerance,4*tolerance), dtype=bool)
    for edge in nx_graph_tm1.edges(node1):
        list_pixel=nx_graph_tm1.get_edge_data(*edge)['pixel_list']
        if (pos_tm1[node1]!=list_pixel[0]).any():
            list_pixel=list(reversed(list_pixel))
#         print(list_pixel[0],pos_tm1[node1],list_pixel[-1])
        for pixel in list_pixel[:20]:
            sparse_cross1[np.array(pixel)-np.array(pos_tm1[node1])+np.array((50,50))]=1
    for edge in nx_graph_t.edges(node2):
        list_pixel=nx_graph_t.get_edge_data(*edge)['pixel_list']
        if (pos_t[node2]!=list_pixel[0]).any():
            list_pixel=list(reversed(list_pixel))
#         print(list_pixel[0],pos_t[node2],list_pixel[-1])
        for pixel in list_pixel[:20]:
#             if np.any(np.array(pixel)-np.array(pos_tm1[node1])+np.array((50,50))>=100):
#                 print(list_pixel[0],pos_t[node2],list_pixel[-1])
            sparse_cross2[np.array(pixel)-np.array(pos_tm1[node1])+np.array((50,50))]=1
    kernel = np.ones((3,3),np.uint8)
    dilation1 = cv2.dilate(sparse_cross1.todense().astype(np.uint8),kernel,iterations = 3)
    dilation2 = cv2.dilate(sparse_cross2.todense().astype(np.uint8),kernel,iterations = 3)
    if show:
        plt.imshow(dilation1)
        plt.imshow(dilation2,alpha=0.5)
        plt.show()
    return(np.linalg.norm(dilation1-dilation2))

def first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,tolerance):
    corresp={}
    ambiguous=set()
    to_remove=set()
    degree_3sup_nodes_tm1 = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)>=3]
    degree_3sup_nodes_t = [node for node in nx_graph_t.nodes if nx_graph_t.degree(node)>=3]
    for node1 in degree_3sup_nodes_tm1:
        mini=np.inf
        for node2 in degree_3sup_nodes_t:
            distance=np.linalg.norm(pos_tm1[node1]-pos_t[node2])
            if distance<mini:
                mini=distance
                identifier=node2
        if mini<tolerance:
            if identifier in corresp.values():
                ambiguous.add(node1)
            corresp[node1]=identifier
        else:
            to_remove.add(node1)
    while len(ambiguous)>0:
        node=ambiguous.pop()
        identifier=corresp[node]
        candidates = [nod for nod in corresp.keys() if corresp[nod]==identifier]
        mini=np.inf
        for candidate in candidates:
            distance=node_dist(candidate,identifier,nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,tolerance)
            if distance < mini:
                right_candidate=candidate
                mini=distance
        for candidate in candidates:
            if candidate!= right_candidate:
                corresp.pop(candidate)
                to_remove.add(candidate)
                ambiguous.discard(candidate)
    return(corresp,to_remove)
    
def relabel_nodes(corresp,nx_graph_t,pos_t):
    invert_corresp={}
    new_pos = {}
    maxi=max(nx_graph_t.nodes)+1
    for key in corresp.keys():
        invert_corresp[corresp[key]]=key
    def mapping(node):
        if node in invert_corresp.keys():
            return(invert_corresp[node])
        else:
            return (maxi+node)
    for node in nx_graph_t.nodes:
        pos=pos_t[node]
        if node in invert_corresp.keys():
            new_pos[invert_corresp[node]]=pos
        else:
            new_pos[maxi+node]=pos
    new_graph=nx.relabel_nodes(nx_graph_t,mapping)
    return(new_pos,new_graph)

def relabel_nodes_downstream(corresp,nx_graph_list,pos_list):
    invert_corresp={}
    new_poss = [{} for i in range(len(nx_graph_list))]
    new_graphs=[]
    all_nodes = set()
    for nx_graph in nx_graph_list:
        all_nodes=all_nodes.union(set(nx_graph.nodes))
    all_nodes=all_nodes.union(set(corresp.keys()))
    all_nodes=all_nodes.union(set(corresp.values()))
    maxi=max(all_nodes)+1
    for key in corresp.keys():
        invert_corresp[corresp[key]]=key
    def mapping(node):
        if node in invert_corresp.keys():
            return(invert_corresp[node])
        else:
            return (maxi+node)
    for i,nx_graph in enumerate(nx_graph_list):
        for node in nx_graph.nodes:
            pos=pos_list[i][node]
            new_poss[i][mapping(node)]=pos
        new_graphs.append(nx.relabel_nodes(nx_graph,mapping,copy=True))
    return(new_graphs,new_poss)


def reduce_labels(nx_graph_list,pos_list):
    new_poss = [{} for i in range(len(nx_graph_list))] 
    new_graphs=[]
    all_node_labels = set()
    node=[node for node in nx_graph_list[0].nodes if node in nx_graph_list[1]][0]
    for nx_graph in nx_graph_list:
        all_node_labels=all_node_labels.union(set(nx_graph.nodes))
    all_node_labels = sorted(all_node_labels)
    def mapping(node):
        return(all_node_labels.index(node))
    for i,nx_graph in enumerate(nx_graph_list):
        for node in nx_graph.nodes:
            pos=pos_list[i][node]
            new_poss[i][mapping(node)]=pos
        new_graphs.append(nx.relabel_nodes(nx_graph,mapping,copy=True))
    node=[node for node in new_graphs[0].nodes if new_graphs[0].degree(node)==3][0]
    for i,nx_graph in enumerate(nx_graph_list):
        if node in new_poss[i].keys():
            print(node,i,new_poss[i][node])
    return(new_graphs,new_poss)

def reconnect_degree_2(nx_graph,pos):
    degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]
    while len(degree_2_nodes)>0:
        node = degree_2_nodes.pop()
        neighbours = list(nx_graph.neighbors(node))
        right_n = neighbours[0]
        left_n = neighbours[1]
        right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
        left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
        if np.any(right_edge[0]!=pos[node]):
            right_edge = list(reversed(right_edge))
        if np.any(left_edge[-1]!=pos[node]):
            left_edge = list(reversed(left_edge))
        pixel_list = left_edge+right_edge[1:]
        info={'weight':len(pixel_list),'pixel_list':pixel_list}
        if right_n!=left_n:
            connection_data=nx_graph.get_edge_data(right_n,left_n)
            if connection_data is None or connection_data['weight']>=info['weight']:
                if not connection_data is None:
                    nx_graph.remove_edge(right_n,left_n)
                nx_graph.add_edges_from([(right_n,left_n,info)])
        nx_graph.remove_node(node)
        degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]
            
def clean_nodes(nx_graph,to_remove,pos):
    nx_graph=deepcopy(nx_graph) #could be removed to speed up
    is_hair = True
    i=0
    while is_hair:
        is_hair=False
        to_remove_possibly=list(to_remove)
        for node in to_remove_possibly:
            neighbours = nx_graph.neighbors(node)
            candidate_to_remove=[]
            weight_candidate=[]
            for neighbour in neighbours:
                if nx_graph.degree(neighbour)==1:
                    is_hair=True
                    candidate_to_remove.append(neighbour)
                    weight_candidate.append(len(nx_graph.get_edge_data(node,neighbour)['pixel_list']))
            if len(candidate_to_remove)>0:
                node_to_remove=candidate_to_remove[np.argmin(weight_candidate)]
                nx_graph.remove_node(node_to_remove)
                i+=1
                if nx_graph.degree(node)==2:
                    to_remove.discard(node)
    print('number removed first phase',i)
    reconnect_degree_2(nx_graph,pos) #could possibly be done faster
    for node in to_remove:
        if node in nx_graph:
            neighbours = list(nx_graph.neighbors(node))
            candidate_to_fuse=[]
            weight_candidate=[]
            for neighbour in neighbours:
                candidate_to_fuse.append(neighbour)
                weight_candidate.append(len(nx_graph.get_edge_data(node,neighbour)['pixel_list']))
            node_to_fuse=candidate_to_fuse[np.argmin(weight_candidate)]
            for neighbour in neighbours:
                right_n = node_to_fuse
                left_n = neighbour
                right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
                left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
                if np.any(right_edge[0]!=pos[node]):
                    right_edge = list(reversed(right_edge))
                if np.any(left_edge[-1]!=pos[node]):
                    left_edge = list(reversed(left_edge))
                pixel_list = left_edge+right_edge[1:]
                info={'weight':len(pixel_list),'pixel_list':pixel_list}
                if right_n!=left_n:
                    connection_data=nx_graph.get_edge_data(right_n,left_n)
                    if connection_data is None or connection_data['weight']>=info['weight']:
                        if not connection_data is None:
                            nx_graph.remove_edge(right_n,left_n)
                        nx_graph.add_edges_from([(right_n,left_n,info)])
            nx_graph.remove_node(node)
            i+=1
    print('number removed second phase',i)
    reconnect_degree_2(nx_graph,pos)
    return(nx_graph)

def orient(pixel_list,root_pos):
    if np.all(root_pos==pixel_list[0]):
        return(pixel_list)
    else:
        return list(reversed(pixel_list))
    
def second_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50,downstream_graphs=[],downstream_pos=[],tolerance=50):
    reconnect_degree_2(nx_graph_t,pos_t)
    corresp,to_remove=first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,tolerance)
    nx_graph_tm1=clean_nodes(nx_graph_tm1,to_remove,pos_tm1)
    downstream_graphs=[nx_graph_t]+downstream_graphs
    downstream_pos=[pos_t]+downstream_pos
    new_graphs,new_poss=relabel_nodes_downstream(corresp,downstream_graphs,downstream_pos)
    pos_t = new_poss[0]
    nx_graph_t = new_graphs[0]
    downstream_pos=new_poss
    downstream_graphs=new_graphs
    corresp_tips={node : node for node in corresp.keys()}
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    ambiguous=set()
    for tip in tips:
        mini=np.inf
        for edge in nx_graph_t.edges:
            pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos_tm1[tip]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
                if distance<mini:
                    mini=distance
                    right_edge = edge
        origin = np.array(orient(nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['pixel_list'],pos_tm1[tip]))
        origin_vector = origin[0]-origin[-1]
        branch=np.array(orient(nx_graph_t.get_edge_data(*right_edge)['pixel_list'],pos_t[right_edge[0]]))
        candidate_vector = branch[-1]-branch[0]
        dot_product = np.dot(origin_vector,candidate_vector)
        if dot_product>=0:
            root=right_edge[0]
            next_node=right_edge[1]
        else:
            root=right_edge[1]
            next_node=right_edge[0]
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        i=0
        loop=[]
        while nx_graph_t.degree(current_node)!=1 and not current_node in nx_graph_tm1.nodes: #Careful : if there is a cycle with low angle this might loop indefinitely but unprobable
            i+=1
            if i>=100:
                print(i,tip,current_node,pos_t[current_node])
            mini=np.inf
            origin_vector = last_branch[0]-last_branch[min(length_id,len(last_branch)-1)]
            unit_vector_origin = origin_vector / np.linalg.norm(origin_vector)
            candidate_vectors=[]
            for neighbours_t in nx_graph_t.neighbors(current_node):
                if neighbours_t!=last_node:
                    branch_candidate=np.array(orient(nx_graph_t.get_edge_data(current_node,neighbours_t)['pixel_list'],pos_t[current_node]))
                    candidate_vector = branch_candidate[min(length_id,len(branch_candidate)-1)]-branch_candidate[0]
                    unit_vector_candidate = candidate_vector / np.linalg.norm(candidate_vector)
                    candidate_vectors.append(unit_vector_candidate)
                    dot_product = np.dot(unit_vector_origin, unit_vector_candidate)
                    angle = np.arccos(dot_product)
                    if angle<mini:
                        mini=angle
                        next_node=neighbours_t
#                     print('angle',dot_product,pos_t[last_node],pos_t[current_node],pos_t[neighbours_t],angle/(2*np.pi)*360)
#!!!bug may happen here if two nodes are direct neighbours : I would nee to check further why it the case, optimal segmentation should avoid this issue.
# This is especially a problem for degree 4 nodes. Maybe fuse nodes that are closer than 3 pixels.
            if i>=100:
                print(mini/(2*np.pi)*360, pos_t[next_node])
                if next_node in loop:
                    break
                else:
                    loop.append(next_node)
            if len(candidate_vectors)<2:
                print("candidate_vectors < 2",nx_graph_t.degree(current_node),pos_t[current_node],[node for node in nx_graph_t.nodes if nx_graph_t.degree(node)==2])
            competitor = np.arccos(np.dot(candidate_vectors[0],-candidate_vectors[1]))
            if mini<competitor:
                current_node,last_node=next_node,current_node
            else:
                break
        if current_node in nx_graph_tm1.nodes:
            if last_node not in nx_graph_tm1.nodes:
                if last_node in corresp_tips.values():
                    ambiguous.add(tip)
                corresp_tips[tip]=last_node
        else:
            if current_node in corresp_tips.values():
                ambiguous.add(tip)
            corresp_tips[tip]=current_node
    while len(ambiguous)>0:
        node=ambiguous.pop()
        identifier=corresp_tips[node]
        candidates = [nod for nod in corresp_tips.keys() if corresp_tips[nod]==identifier]
        mini=np.inf
        for candidate in candidates:
            distance=np.linalg.norm(pos_tm1[candidate]-pos_t[identifier])
            print(identifier,distance)
            if distance < mini:
                right_candidate=candidate
                mini=distance
        for candidate in candidates:
            if candidate!= right_candidate:
                corresp_tips.pop(candidate)
                ambiguous.discard(candidate)
    new_graphs,new_poss=relabel_nodes_downstream(corresp_tips,downstream_graphs,downstream_pos)
    downstream_pos=new_poss
    downstream_graphs=new_graphs
#     print("second relabeling")
#     print(len(nx_graph_tm1.nodes),len(new_graphs[0].nodes))
    new_graphs,new_poss=reduce_labels([nx_graph_tm1]+downstream_graphs,[pos_tm1]+downstream_pos)
#     print("third relabeling")
#     print(len(new_graphs[0].nodes),len(new_graphs[1].nodes))
    return(new_graphs,new_poss)
                

def whole_movement_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50):
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    tip_origin={tip : [tip]  for tip in tips}
    pixels_from_tip={tip : [] for tip in tips}
    for number,tip in enumerate(tips):
#         print('tip',pos_tm1[tip],tip)
        labeled=[]
        if number%100==0:
            print(number/len(tips))
        mini=np.inf
        for edge in nx_graph_t.edges:
            pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos_tm1[tip]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
                if distance<mini:
                    mini=distance
                    right_edge = edge
        origin = np.array(orient(nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['pixel_list'],pos_tm1[tip]))
        origin_vector = origin[0]-origin[-1]
        branch=np.array(orient(nx_graph_t.get_edge_data(*right_edge)['pixel_list'],pos_t[right_edge[0]]))
        index_nearest_pixel=np.argmin(np.linalg.norm(branch-np.array(pos_tm1[tip]),axis=1))
        candidate_vector = branch[-1]-branch[0]
        dot_product = np.dot(origin_vector,candidate_vector)
#         if tip==5260:
#             print(list(branch[index_nearest_pixel:]))
#             print(list(branch[:index_nearest_pixel]))
        if dot_product>=0:
            root=right_edge[0]
            next_node=right_edge[1]
            pixels_from_tip[tip].append(list(branch[index_nearest_pixel:]))
        else:
            root=right_edge[1]
            next_node=right_edge[0]
            pixels_from_tip[tip].append(list(reversed(list(branch[:index_nearest_pixel]))))
        #Could improve the candidate vector by chosing pixel around the forme tip but this identification should be rather unambiguous
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        def label_node_recursive(last_node,current_node,corresp_label,labeled):
            if not current_node in labeled and not current_node in nx_graph_tm1.nodes:
                corresp_label[tip].append(current_node)
                labeled.append(current_node)
                pixel_list=nx_graph_t.get_edge_data(last_node,current_node)['pixel_list']
                pixels_from_tip[tip].append(pixel_list)
                if nx_graph_t.degree(current_node)>=3:
                    for neighbour_t in nx_graph_t.neighbors(current_node): 
                        if neighbour_t!=last_node:
                            label_node_recursive(current_node,neighbour_t,corresp_label,labeled)
        label_node_recursive(last_node,current_node,tip_origin,labeled)
    return(tip_origin,pixels_from_tip)

def shift(skeleton1,skeleton2):
    skeleton1_dilated = dilate(dilate(skeleton1)).astype(np.float)
    skeleton2_dilated = dilate(dilate(skeleton2)).astype(np.float)
    def distance(shift):
        distance=0
#         print(shift)
        for pixel in skeleton1_dilated.keys():
#             print(pixel[0]+shift[0],pixel[1]+shift[1])
            if (skeleton2_dilated.shape[0]>np.ceil(pixel[0]+shift[0])>=0 and skeleton2_dilated.shape[1]>np.ceil(pixel[1]+shift[1])>=0):
                shifted_pixel = (int(pixel[0]+shift[0]),int(pixel[1]+shift[1]))
                shifted_pixel_next = (np.ceil(pixel[0]+shift[0]),np.ceil(pixel[1]+shift[1]))
#                 print(shifted_pixel)
                prop=1/2*(pixel[0]+shift[0]-int(pixel[0]+shift[0])+pixel[1]+shift[1]-int(pixel[1]+shift[1]))
                float_value=(1-prop)*skeleton2_dilated[shifted_pixel[0],shifted_pixel[1]]+prop*(skeleton2_dilated[shifted_pixel_next[0],shifted_pixel_next[1]])
                distance+=abs(skeleton1_dilated[pixel]-float_value)
            else:
                distance+=1
#         for pixel in skeleton2_dilated.keys():
#             if (skeleton2_dilated.shape[0]>pixel[0]-shift[0]>=0 and skeleton2_dilated.shape[1]>pixel[1]-shift[1]>=0):
#                 shifted_pixel = (int(pixel[0]-shift[0]),int(pixel[1]-shift[1]))
#                 distance+=abs(skeleton1_dilated[shifted_pixel[0],shifted_pixel[1]]^skeleton2_dilated[pixel])
#             else:
#                 distance+=1
#         print(distance)
        return distance
    return(minimize(distance,np.array([10,10]), method='nelder-mead',options={'xatol': 1, 'disp': True,'fatol':0.1}))